In [1]:
## Imports
from __future__ import print_function

import torch
from torch import optim
from torch.utils.data import DataLoader
import torch.nn as nn

from unet.unet_model import RegressorUNet
from unet.unet_parts import SegmentationRegressionLoss

from utils.dataset import CrossValDataset
from utils.model_eval import dice_coefficient
from utils.data import RegressorDataset

from torch.utils.tensorboard import SummaryWriter

import numpy as np

In [2]:
## Device
if torch.cuda.is_available():
    device = 'cuda'
    print('Using GPU')
else:
    device = 'cpu'
    print('Using CPU')

Using GPU


In [3]:
## Load Model
modelpath = "D:/autopos/nets/unet-crop-save3_9.pth"
pretrained_model = torch.load(modelpath, map_location='cpu')

In [4]:
## Dataset
groupPath  = "D:/autopos/train"
originPath = "D:/autopos/train/origin.csv"
batch_size = 4

## Training
n_channels = 1
n_classes = 13
n_regress = 2
# n_bottle = 256
# n_hidden = [512,256,128,64,32,16]
# n_hidden = [512,256]
# n_hidden = [1024,1024,512,512,256,256]
n_reduce = [64,128]
n_hidden = [256,256,256]
img_size = 512

learning_rate = 1e-3
weight_decay  = 1e-8
momentum      = 0.9

num_epochs    = 41

##Cropping
D = 350
N = 512
xc=0
yc=0
centermode = 'rand'
mu = np.array([0,0])
sigma= 50

In [ ]:
## Train
fullDirList = np.array([groupPath+"/group_00",groupPath+"/group_01",groupPath+"/group_02",groupPath+"/group_03",groupPath+"/group_04"])
inds = np.arange(len(fullDirList))
for partition in range(len(fullDirList)):
# for partition in range(1):
    trainInd = np.delete(inds,[partition])
    testInd  = inds[partition]
    trainDirs = fullDirList[trainInd]
    testDir   = [fullDirList[testInd]]
    
    trainDataset = RegressorDataset([x + "/img" for x in trainDirs], [x + "/seg" for x in trainDirs], originPath,D=D,N=N,xc=xc,yc=yc,centermode=centermode,mu=mu,sigma=sigma)
    testDataset  = RegressorDataset([x + "/img" for x in testDir],   [x + "/seg" for x in testDir],   originPath,D=D,N=N,xc=xc,yc=yc,centermode=centermode,mu=mu,sigma=sigma)
    
    trainLoader  = DataLoader(trainDataset,batch_size=batch_size)
    testLoader   = DataLoader(testDataset,batch_size=batch_size)
    
    # Define Net
#     net = RegressorUNet(n_channels=n_channels, n_classes=n_classes,n_regress=n_regress,n_hidden=n_hidden,n_bottle=n_bottle,img_size=img_size)
#     net.to(device=device)
#     net.apply_state_dict(pretrained_model)
    
#     net = RegressorNet(n_channels=n_channels, n_regress=n_regress,n_hidden=n_hidden,n_bottle=n_bottle,img_size=img_size)
#     net = RegressorNet(n_channels=n_channels, n_regress=n_regress,n_hidden=n_hidden,img_size=img_size)
    net = RegressorUNet(n_channels,n_classes,n_reduce,n_hidden,n_regress,img_size)
    print(net)
    net.apply_state_dict(pretrained_model)
    net.to(device=device)
    
    
    print("Successfully loaded pretrained model")
    
    # Optimizer
    optimizer  = optim.RMSprop(net.parameters(),lr = learning_rate, weight_decay = weight_decay, momentum = momentum)
#     critereon  = SegmentationRegressionLoss()  
    critereon = nn.MSELoss()
#     lossweight = 100 
    
    #Tensorboard
    writer = SummaryWriter('runs/regress_model5_partition_'+str(partition))
    
    for epoch in range(num_epochs):
        net.train()
        epoch_train_loss = 0
        for trainIndex,batch in enumerate(trainLoader,0):
            imgs = batch['img']
#             segs_gt = batch['seg']
            regs_gt = batch['org']
            
            imgs = imgs.to(device=device, dtype=torch.float32)
#             segs_gt = segs_gt.to(device=device, dtype = torch.long)
            regs_gt = regs_gt.to(device=device, dtype = torch.float32)
            
#             segs_pr,regs_pr = net(imgs)
            regs_pr = net(imgs)
            
#             loss = critereon(segs_pr,regs_pr,segs_gt,regs_gt,lossweight)
            loss = critereon(regs_pr,regs_gt)
            
            epoch_train_loss += loss.item()/len(trainLoader)
            
            regs_err = regs_pr-regs_gt
            print(regs_err)
            
            optimizer.zero_grad()
            loss.backward()
#             nn.utils.clip_grad_value_(net.parameters(),0.1)
            optimizer.step()
            
            print("Train: " + str(loss.item()))
            
        writer.add_scalar('Training Loss',
                        epoch_train_loss,
                        epoch)
        
        net.eval()
        epoch_test_loss = 0
        for testIndex,batch in enumerate(testLoader,0):
            imgs = batch['img']
#             segs_gt = batch['seg']
            regs_gt = batch['org']
            
            imgs = imgs.to(device=device, dtype=torch.float32)
#             segs_gt = segs_gt.to(device=device, dtype = torch.long)
            regs_gt = regs_gt.to(device=device, dtype = torch.float32)
            
#             segs_pr,regs_pr = net(imgs)
            regs_pr = net(imgs)
            
#             loss = critereon(segs_pr,regs_pr,segs_gt,regs_gt,lossweight)
            loss = critereon(regs_pr,regs_gt)
    
            epoch_test_loss += loss.item()/len(testLoader)
            
            regs_err = regs_pr-regs_gt
            print(regs_err)
            
            print("Test: " + str(loss.item()))
            
        writer.add_scalar('Validation Loss',
                        epoch_test_loss,
                        epoch)
        
#         savefile = "D:\\autoseg\\nets\\unet-save" + str(partition) + "_" + str(epoch) + ".pth"
#         torch.save(net.state_dict(),savefile)


RegressorUNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-0

Train: 3606859.0
tensor([[ -981.9326, -1371.0845],
        [ -429.5708,  -813.5112],
        [-1512.1472, -1844.8693],
        [-1467.8289, -1796.7218]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 1845407.375
tensor([[1004.2398,   59.6421],
        [1470.7819,  442.0633],
        [1762.7042,  734.6041],
        [1213.9565,  164.5981]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 1064778.25
tensor([[3760.2322, 1711.4484],
        [5604.1387, 1470.6372],
        [3942.5359, 2153.2178],
        [4360.3433, 2028.6736]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 11743199.0
tensor([[-4464.3296, -3248.1340],
        [-5351.9897, -3982.2139],
        [-5485.4141, -4065.5339],
        [-4302.0396, -3118.7170]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 18729338.0
tensor([[ -397.2887,  -625.2198],
        [  -71.3799,   -94.6945],
        [ -332.6841, -1086.5721],
        [ -160.0299,  -140.9780]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 237450.296875
tensor([[ 142.1

Train: 3160.29931640625
tensor([[  -4.8857,  118.2233],
        [  20.6406,  137.6860],
        [ 167.9034, -170.2905],
        [ -98.6968, -135.7072]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 14841.5
tensor([[-31.7575,  74.9860],
        [ 27.9841, 131.4890],
        [ 12.7735, 111.5299]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 6217.66552734375
tensor([[ -15.7053,   82.9492],
        [   2.5181, -360.3015],
        [-573.1040,  661.8323],
        [  91.0449,  -33.2163]], device='cuda:0', grad_fn=<SubBackward0>)
Test: 114101.671875
tensor([[-68.7996,  89.6611],
        [155.3784,  -8.6091],
        [132.6371, -31.3557],
        [ 93.8829, 111.9342]], device='cuda:0', grad_fn=<SubBackward0>)
Test: 9613.51171875
tensor([[  8.7222,  29.4734],
        [ 54.7944, 113.2808],
        [-58.5682,  31.4787],
        [ 63.9812, -88.0890]], device='cuda:0', grad_fn=<SubBackward0>)
Test: 4131.765625
tensor([[  47.9550,   79.9591],
        [ -35.7481,  124.2819],
        [-128.3585,

Train: 3508.51171875
tensor([[ 51.6202, 113.3679],
        [-73.5327, -19.7308],
        [ 11.5643, -35.6211],
        [ 13.9910, 111.2031]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 4409.71923828125
tensor([[-28.8376,  63.0252],
        [-66.0716, 104.9672],
        [ 42.2105,  61.3093],
        [ 26.2297, 142.5937]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 5843.60888671875
tensor([[-56.9508, -40.2290],
        [ 47.9605,  -6.0887],
        [-63.5646,   6.2784],
        [-84.3858, -96.3307]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 3459.93798828125
tensor([[ -91.1277, -127.4861],
        [ -12.8293,  -52.7941],
        [ -21.0077, -136.6928],
        [ -25.0717,  -17.5286]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 5946.3583984375
tensor([[-21.4345, 118.4451],
        [  6.1305,  42.4051],
        [-32.5323,  69.3937],
        [ 29.2862, -72.7478]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 3543.526123046875
tensor([[ 24.3295,  28.8152],
        [ -

tensor([[ -5.6684, -19.0024],
        [-31.2622,  41.1608],
        [  6.1949,  95.7231],
        [  6.4910,  17.5687]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 1577.105224609375
tensor([[ 27.4917, -25.8106],
        [ 23.7352,  44.0488],
        [ 50.7577,  33.0084],
        [ -2.7826,  60.8791]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 1413.1925048828125
tensor([[-0.4385, 36.2247],
        [31.2021, 29.5581],
        [42.5902, 55.5927]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 1344.02490234375
tensor([[ -6.1607,  19.5555],
        [-11.8167,  38.2980],
        [ 19.3548,  99.9610],
        [-41.0768,  16.2280]], device='cuda:0', grad_fn=<SubBackward0>)
Test: 1793.02587890625
tensor([[-28.9466, -56.4688],
        [ 35.2710,  83.3095],
        [  0.9482, -58.7166],
        [  4.9319,   7.5195]], device='cuda:0', grad_fn=<SubBackward0>)
Test: 1967.568603515625
tensor([[  8.4818,  44.3542],
        [ 44.6242, 154.4050],
        [ 46.7205,  79.6759],
        [-58.3

...

Train: 3170.62646484375
tensor([[42.8243,  7.6867],
        [65.4744, 47.9813],
        [22.1228, 53.6389],
        [44.2550, 35.1036]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 1879.927734375
tensor([[64.2243, 37.2394],
        [ 6.1042, -4.7740],
        [42.7309, 25.4485],
        [-2.0937, 23.7282]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 1076.568603515625
tensor([[-30.6759, -14.1844],
        [-30.2457,  19.0524],
        [ 27.6546,   0.1506],
        [ 52.3923,  12.4393]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 760.5619506835938
tensor([[-45.9891, -84.9284],
        [-18.3241, -54.8825],
        [  8.7775, -33.9440],
        [-30.0457, -87.8084]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 2814.74951171875
tensor([[-16.0750,  40.5259],
        [-58.6805,  44.9836],
        [-29.5376,   8.9930],
        [ 42.8171,  47.1932]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 1547.69091796875
tensor([[-44.1216,  35.5171],
        [ 22.3918,  52.9416],
  

Train: 7257.61328125
tensor([[ 42.5396, 102.9549],
        [ 19.3191,  66.4928],
        [  9.0931, 110.3704],
        [ 48.3240, 145.2840]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 6613.8486328125
tensor([[-40.6626, 142.4924],
        [ 27.1684, 121.6536],
        [ 15.0888, 134.8999],
        [ 49.7771,  77.1786]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 8044.40087890625
tensor([[-31.3675, 109.6308],
        [  6.8826,  96.3223],
        [ 42.9893, 137.6539],
        [-32.6127, 123.2946]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 7423.75
tensor([[ -0.6495,  76.5243],
        [  7.1439,  89.4170],
        [  9.8670, 102.9679],
        [ 23.7359, 109.3292]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 4639.8564453125
tensor([[-4.7907e-02,  1.2985e+01],
        [ 3.0986e+01,  1.5821e+02],
        [-6.9749e+00,  7.3872e+01],
        [-4.6391e+01, -2.5759e+01]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 4310.03271484375
tensor([[ -91.8149,  -86.3577],
    

tensor([[-83.2386,   3.4107],
        [ 47.8053,   6.1969],
        [-51.4169,  34.3693],
        [ 28.8500,  42.3227]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 1964.065185546875
tensor([[-14.2031,  28.5640],
        [ 20.0262,  24.6216],
        [ 37.0373,  12.2601],
        [ 49.6693,  22.8439]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 816.9822998046875
tensor([[-50.0854,  39.5917],
        [ 41.8170,  29.2019],
        [-18.4312,  55.9509],
        [ -4.1415,  -5.5437]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 1274.4456787109375
tensor([[-53.9661,  17.1742],
        [-26.4179, -10.1713],
        [-52.9562,  -4.0754],
        [-21.5350,  -3.5174]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 913.219482421875
tensor([[  10.6303,  -42.5783],
        [  45.8519,  -19.6620],
        [ -13.0032,  -30.0720],
        [  42.7008, -153.3509]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 3853.5234375
tensor([[-48.8628,  13.0370],
        [-66.1678,  20.9092],
  

Train: 38161.50390625
tensor([[ 93.2744,  80.6079],
        [  3.7370,  66.3174],
        [ 82.0089,  27.2049],
        [103.4253,  67.2923]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 5287.5419921875
tensor([[ 56.2271, -11.1476],
        [ 42.1304,  37.3004],
        [ 99.0219,  57.2335],
        [ 17.4592,  58.6720]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 2910.036376953125
tensor([[-4.9638e+01, -6.6064e-01],
        [-2.3625e+01,  4.9545e+00],
        [-2.6286e+00, -2.6826e+01],
        [-4.5094e-02, -3.4143e+01]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 617.413818359375
tensor([[-132.7544,  -51.5952],
        [-122.3893,  -46.5279],
        [-128.3807,   26.3030],
        [-100.5264,  -20.5340]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 8141.30224609375
tensor([[ 64.1861,  -0.5710],
        [ 23.9636, -10.7487],
        [ 28.5765, -59.7361],
        [ 20.8339, -29.0589]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 1309.18212890625
tensor([[ 50.407

Train: 835.68408203125
tensor([[ 43.9177, -27.1872],
        [-48.8854, -47.8470],
        [-24.0311, -95.1634],
        [  4.0032, -61.7846]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 2601.7431640625
tensor([[-64.0061, -13.8623],
        [-25.6374,  -0.3358],
        [-10.5626,   8.6095],
        [-21.1512, -45.3803]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 954.8463134765625
tensor([[-48.9291,  -4.8984],
        [-25.2382,  19.4296],
        [-20.6566,  36.4375],
        [ -9.5548,  25.4800]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 740.9303588867188
tensor([[ 36.9446,  20.8278],
        [ 38.8284,  24.1836],
        [-40.4974,   6.3909],
        [-13.9045,  10.5165]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 734.5012817382812
tensor([[  4.2650,   9.5310],
        [-13.8607,  49.3213],
        [ 20.1020,  39.4591],
        [ 70.5427,  42.8104]], device='cuda:0', grad_fn=<SubBackward0>)
Train: 1437.98193359375
tensor([[-17.9597, -31.3487],
        [ 34.95